In [ ]:
import numpy as np
def get_valid_bin_indices(item: float, bins: np.ndarray) -> np.ndarray:
  """Returns indices of bins in which item can fit."""
  return np.nonzero((bins - item) >= 0)[0]


def online_binpack(
    items: tuple[float, ...], bins: np.ndarray
, priority) -> tuple[list[list[float, ...], ...], np.ndarray]:
  """Performs online binpacking of `items` into `bins`."""
  # Track which items are added to each bin.
  packing = [[] for _ in bins]
  # Add items to bins.
  for item in items:
    # Extract bins that have sufficient space to fit item.
    valid_bin_indices = get_valid_bin_indices(item, bins)
    # Score each bin based on heuristic.
    priorities = priority(item, bins[valid_bin_indices])
    # Add item to bin with highest priority.
    best_bin = valid_bin_indices[np.argmax(priorities)]
    bins[best_bin] -= item
    packing[best_bin].append(item)
  # Remove unused bins from packing.
  packing = [bin_items for bin_items in packing if bin_items]
  return packing, bins


# @funsearch.run
def evaluate(instances: dict, priority) -> float:
  """Evaluate heuristic function on a set of online binpacking instances."""
  # List storing number of bins used for each instance.
  num_bins = []
  # Perform online binpacking for each instance.
  for name in instances:
    instance = instances[name]
    capacity = instance['capacity']
    items = instance['items']
    # Create num_items bins so there will always be space for all items,
    # regardless of packing order. Array has shape (num_items,).
    bins = np.array([capacity for _ in range(instance['num_items'])])
    # Pack items into bins and return remaining capacity in bins_packed, which
    # has shape (num_items,).
    _, bins_packed = online_binpack(items, bins, priority)
    # If remaining capacity in a bin is equal to initial capacity, then it is
    # unused. Count number of used bins.
    num_bins.append((bins_packed != capacity).sum())
  # Score of heuristic function is negative of average number of bins used
  # across instances (as we want to minimize number of bins).
  return -np.mean(num_bins)

In [ ]:
def l1_bound(items: tuple[int, ...], capacity: int) -> float:
  """Computes L1 lower bound on OPT for bin packing.

  Args:
    items: Tuple of items to pack into bins.
    capacity: Capacity of bins.

  Returns:
    Lower bound on number of bins required to pack items.
  """
  return np.ceil(np.sum(items) / capacity)


def l1_bound_dataset(instances: dict) -> float:
  """Computes the mean L1 lower bound across a dataset of bin packing instances.

  Args:
    instances: Dictionary containing a set of bin packing instances.

  Returns:
    Average L1 lower bound on number of bins required to pack items.
  """
  l1_bounds = []
  for name in instances:
    instance = instances[name]
    l1_bounds.append(l1_bound(instance['items'], instance['capacity']))
  return np.mean(l1_bounds)

def get_opt_num_bins(instance):
    return l1_bound_dataset(instance)

#### 搭建两个llm, 其中一个负责产生代码, 其中一个负责生成intuition.

In [ ]:
# 这次搭建的databse是用num_iter作为key的.

import importlib
def update_function(code_response: str, intuition_response: str , instances: dict,
                    database: dict, num_item: int,
                    eval_fun, parents: list) -> dict:
    #模型生成的code, intuition_response 来自于不同的两个步骤.
    module_name = f"my_function_{num_item}"
    with open(f'./cody/{module_name}.py', 'w') as file:
        file.write('import numpy as np \n' + code_response)
    module = importlib.import_module('cody.'+module_name)
    priority = getattr(module, 'priority')
    opt_num_bins = get_opt_num_bins(instances)
    avg_num_bins = -eval_fun(instances, priority)
    score = (avg_num_bins - opt_num_bins) / opt_num_bins
    scores = database.keys()
    if str(score) in scores:
        print('出现重复')
        database[f"{num_item}"] = {"is_repeat": True,
                                   "score": score, "parents": parents,
                                   "code": code_response, "intuition": intuition_response}
    else: 
        database[f"{num_item}"] = {"is_repeat": False,
                                   "score": score, "parents": parents,
                                   "code": code_response, "intuition": intuition_response}
    return database

In [4]:
import random
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks import get_openai_callback
''' 设计思路是:
1. 给gpt_1的prompt喂两个intuition, 
2. 然后把gpt_1生成的intuition喂给gpt_2
'''

def opt(llm_code, llm_response, 
        code_prompt: str, intuition_prompt: str, Meta_prompt : str,
        input_database: dict, expect_num=100, out_database = {},
        pick_type = 'top', top = 5) -> dict:
    intuition_prompt_template = ChatPromptTemplate(messages=[SystemMessagePromptTemplate.from_template(Meta_prompt),
                                              HumanMessagePromptTemplate.from_template(intuition_prompt)], 
                                              input_variables=['intuition1','intuition2'])
    for num_item in range(expect_num):
        if pick_type == 'random':
            key0, key1 = random.sample(list(input_database.keys()), 2)
        elif pick_type == 'top':
            database_ordered = sorted(input_database.items(), key = lambda x: x[1]["score"])[:top]
            candidate = random.sample(database_ordered, 2)
            key0, key1 = candidate[0][0], candidate[1][0]
            
        
        value0,value1 = input_database[key0], input_database[key1]
        #code1, code2 = value0['code'], value1['code'] 似乎code没必要写出来...
        intuition1, intuition2 = value0['intuition'], value1['intuition']
        parents = [key0, key1]
        prompt_message = intuition_prompt_template.format_prompt(#这里可以修改, 应该修改一下1
            intuition1 = intuition1,
            intuition2 = intuition2
        )
        with get_openai_callback() as cb:
            answer = llm_code()
        

In [7]:
dct = {1: {'a':1, 'b':2}, 2: {'a':3, 'b':4}}
dct = sorted(dct.items(), key = lambda x: x[1]["a"], reverse=True)
dct

[(2, {'a': 3, 'b': 4}), (1, {'a': 1, 'b': 2})]